In [53]:

import cv2
import matplotlib.pyplot as plt
import imutils
import glob
from PIL import Image
import numpy as np
from torchvision.models.resnet import ResNet50_Weights
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from keras.models import load_model
import joblib

In [58]:
def gaussian(img):
    smoothed_img = cv2.GaussianBlur(img, (3,3), 0)
    return smoothed_img

def threshold(img):
  
    thresh_img = cv2.threshold(img, 50, 255, cv2.THRESH_TOZERO)[1]
    return thresh_img
def erode(img):
    smoothed_img = cv2.erode(img, None, iterations=1)
    return smoothed_img
def dilate(img):
    smoothed_img = cv2.dilate(img, None, iterations=1)
    return smoothed_img
def contour(source):       
    smoothed_img = cv2.GaussianBlur(source, (3,3), 0)
    gray = cv2.cvtColor(smoothed_img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_TOZERO)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    img_cnt = cv2.drawContours(source.copy(), [c], -1, (0, 255, 255), 4)
    img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
    img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
    img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
    img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)


    ADD_PIXELS = 0
    new_img = source[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    return new_img


def watershed(img):  
   
    source=img.copy()
    shifted = cv2.pyrMeanShiftFiltering(img, 30, 30)

    gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)

    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)


    sure_bg = cv2.dilate(opening,kernel,iterations=3)


    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.1*dist_transform.max(),255,0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)


    ret, markers = cv2.connectedComponents(sure_fg)


    markers = markers+1

    markers[unknown==255] = 0
    markers = cv2.watershed(img,markers)
    img[markers == -1] = [255,255,0]
    img[markers == 1] = [0,0,0]
    

    dst = cv2.addWeighted(source, 0.5, img, 1.5, 0)
  
    
    return dst
def feature_extractor(img):
    transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
    image = transform(img)

# Define the feature extractor
    backbone = torchvision.models.detection.backbone_utils.resnet_fpn_backbone(backbone_name='resnet50', weights='ResNet50_Weights.DEFAULT')




# Extract features for each image and save to a file
    features = []
    
    
    # Extract features
    with torch.no_grad():
        features.append(backbone(image)['0'].squeeze().cpu().numpy())

# Save the features to a file
    features = np.array(features)
    np.save('unlabeledfeatures.npy', features)
def predict(data):
# Load the classifier and predict unlabeled samples
    

    clf = joblib.load('classifier.pkl')
    X_unlabeled = np.load('unlabeledfeatures.npy') # shape: (num_samples_unlabeled, height, width, num_channels)
    X_unlabeled = np.reshape(X_unlabeled, (X_unlabeled.shape[0], -1)) # shape: (num_samples_unlabeled, num_features)
    y_pred = clf.predict(X_unlabeled)
    return y_pred
if __name__ == '__main__':
    path = glob.glob("unlabeled\*.png")
    
    c=0


    for file in path:
        source=cv2.imread(file)
        source1=gaussian(source)
        source2=threshold(source1)
        source3=erode(source2)
        source4=dilate(source3)
        source5=contour(source)
        source6=watershed(source5)
       
        
        pil_image = Image.fromarray(np.uint8(source6 * 255))


        source7=feature_extractor(pil_image)
        file_path = 'unlabeledfeatures.npy'
        data = np.load(file_path)
        classe=predict(data)
        print(classe)
        if(classe==1.):
            cv2.imwrite(f'labeledm/malignant_new{c}.png',source)
        else:
            cv2.imwrite(f'labeledb/beningn_new{c}.png',source)
        c+=1
        cv2.waitKey(0)

        cv2.destroyAllWindows()





   

[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
